In [33]:
import django
import os
import json
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_settings'
django.setup()

In [34]:
import pandas as pd
import requests
from io import BytesIO

In [35]:
from data_management.models import *

In [36]:
CroppedImage.objects.select_related('origin_source')

<QuerySet [<CroppedImage: CroppedImage object (178)>, <CroppedImage: CroppedImage object (179)>, <CroppedImage: CroppedImage object (180)>, <CroppedImage: CroppedImage object (181)>, <CroppedImage: CroppedImage object (182)>, <CroppedImage: CroppedImage object (183)>, <CroppedImage: CroppedImage object (184)>, <CroppedImage: CroppedImage object (185)>, <CroppedImage: CroppedImage object (186)>, <CroppedImage: CroppedImage object (187)>, <CroppedImage: CroppedImage object (188)>, <CroppedImage: CroppedImage object (189)>, <CroppedImage: CroppedImage object (190)>, <CroppedImage: CroppedImage object (191)>, <CroppedImage: CroppedImage object (192)>, <CroppedImage: CroppedImage object (193)>, <CroppedImage: CroppedImage object (194)>, <CroppedImage: CroppedImage object (195)>, <CroppedImage: CroppedImage object (196)>, <CroppedImage: CroppedImage object (197)>, '...(remaining elements truncated)...']>

In [37]:
class DataExtractSave():
    def __init__(self):
        self.left = []
        self.top = []
        self.right = []
        self.bottom = []
        self.origin_url = []
        self.width = []
        self.height = []
    
    def get_image_size(self, resp):
        from PIL import Image
        byteImgIO = BytesIO()
        byteImg = Image.open(BytesIO(resp.content))
        byteImg.save(byteImgIO, "JPEG")
        byteImgIO.seek(0)
        byteImg = byteImgIO.read()
        dataBytesIO = BytesIO(byteImg)
        image = Image.open(dataBytesIO)
        image = image.convert('RGB')
        width, height = image.size
        return width, height
    
    def get_data_and_append(self, obj, url):
        self.origin_url.append(url)
        resp = requests.get(url)
        w, h = self.get_image_size(resp)
        self.width.append(w)
        self.height.append(h)
        self.left.append(obj.left*w)
        self.top.append(obj.top*h)
        self.right.append(obj.right*w)
        self.bottom.append(obj.bottom*h)
    
    def save_to_csv(self,  filename):
        dic = {}
        dic['left'] = self.left
        dic['top'] = self.top
        dic['right'] = self.right
        dic['bottom'] = self.bottom
        dic['origin_url'] = self.origin_url
        dic['width'] = self.width
        dic['height'] = self.height
    
        test_data_pd=pd.DataFrame.from_dict(dic)
        print(filename)
        test_data_pd.to_csv(filename, mode='w', index=False)
        print('downloaded')
        
#     def save_data_from_cropped(self, filename, n):
#         print(n)
        
#         if not n:
#             queryset = CroppedImage.objects.filter(origin_source__image_url__isnull=False)
#         else:
#             N = int(n)
#             queryset = CroppedImage.objects.filter(origin_source__image_url__isnull=False)[:N]
#         i = 0
#         for c in queryset:
#             i += 1
#             print(i)
#             print(c)
#             if c.origin_source.image:
#                 url = c.origin_source.image.url
#                 self.get_data_and_append(c, url)
#         self.save_to_csv(filename)
#         print('DONE!')
        
#     def save_data_from_s3_temp(self, filename, n):       
#         if not n:
#             queryset = CroppedImage.objects.filter(origin_source__image="")
#         else:
#             N = int(n)
#             queryset = CroppedImage.objects.filter(origin_source__image="")[:N]
#             print(queryset)
#         i = 0
#         for c in queryset:
#             i +=1
#             print(i)
#             print(c)
#             if c.origin_source.s3_image_url:
#                 url = c.origin_source.s3_image_url
#                 self.get_data_and_append(c, url)
#         self.save_to_csv(filename)
#         print('DONE!')
        
    def save_data_to_csv(self, filename, queryset):       
        print(queryset.count())
        i = 0
        for c in queryset:
            i +=1
            if i%50==0:
                print(i,'/ ',c)
            if c.origin_source.image:
                url = c.origin_source.image.url
                self.get_data_and_append(c, url)
            elif c.origin_source.s3_image_url:
                url = c.origin_source.s3_image_url
                self.get_data_and_append(c, url)
        self.save_to_csv(filename)
        print('DONE!')

In [9]:
# save_data_from_cropped : 크로링 박싱후 저장한 데이터 csv로 저장
# save_data_from_cropped('저장할 파일이름', 쿼리셋 개수) --> 쿼리셋 개수는 비워두면 전체에 대해 다운로드
# Ex: save_data_from_cropped("crop_download_test.csv","")

In [11]:
downloadmanager1= DataExtractSave()
queryset1 = queryset = CroppedImage.objects.filter(origin_source__image_url__isnull=False)

In [ ]:
downloadmanager1.save_data_to_csv("crop_download_test.csv", queryset1)

In [13]:
# save_data_from_s3_temp : s3업로드 파일 박싱후 저장한 데이터 csv로 저장
# save_data_from_s3_temp('저장할 파일이름', 쿼리셋 개수) --> 쿼리셋 개수는 비워두면 전체에 대해 다운로드
# Ex: save_data_from_s3_temp("s3upload_download_test.csv","")

In [14]:
downloadmanager2= DataExtractSave()
queryset2 = CroppedImage.objects.filter(origin_source__image="")

In [ ]:
downloadmanager2.save_data_to_csv("s3upload_download_test.csv", queryset2)

In [ ]:
# image review (몸체만 정확히 박싱된 데이터) extract

In [38]:
downloadmanager3= DataExtractSave()
queryset3 = CroppedImage.objects.filter(origin_source__image_review=True)

In [39]:
downloadmanager3.save_data_to_csv("20191021_tight_boxed_data.csv", queryset3)

1000
50 /  CroppedImage object (7895)
100 /  CroppedImage object (7945)
150 /  CroppedImage object (7996)
200 /  CroppedImage object (8047)
250 /  CroppedImage object (8097)
300 /  CroppedImage object (8147)
350 /  CroppedImage object (8197)
400 /  CroppedImage object (8248)
450 /  CroppedImage object (8298)
500 /  CroppedImage object (8348)
550 /  CroppedImage object (8398)
600 /  CroppedImage object (8448)
650 /  CroppedImage object (8498)
700 /  CroppedImage object (8548)
750 /  CroppedImage object (8598)
800 /  CroppedImage object (8648)
850 /  CroppedImage object (8698)
900 /  CroppedImage object (8748)
950 /  CroppedImage object (8798)
1000 /  CroppedImage object (9011)
20191021_tight_boxed_data.csv
downloaded
DONE!


In [20]:
CroppedImage.objects.filter(origin_source__image="").exclude(origin_source__image="").count()

0

In [6]:
CroppedImage.objects.filter(origin_source__image_url__isnull=False).count()

8565

In [10]:
CroppedImage.objects.all().count()

8565

In [31]:
CroppedImage.objects.filter(origin_source__image_review=True).count()

1000